
# Bioinformatic analysis of the role of GemC1 in the regulation of transcription and chromatin dynamics by BAF complex during differentiation of multiciliated ependymal cells

**Project:** Bioinformatic analysis of the role of GemC1 in chromatin dynamics.  
**Assay:** CUT&RUN (Brg1)  
**Organism:** *Mus musculus* (mm10)

This notebook documents the full CUT&RUN bioinformatics pipeline used in my master's thesis project titled “Bioinformatic analysis of the role of GemC1 in the regulation of transcription and chromatin dynamics by BAF complex during differentiation of multiciliated ependymal cells”.

The full text can be found in the following link: https://nemertes.library.upatras.gr/items/1781a0e5-00d6-480e-abc1-7257d588e0ce


## 1. Software and Environment

### Linux tools
- Bowtie2 (>= 2.3.4.3)
- samtools (>= 1.10)
- bedtools (>= 2.29.1)
- SEACR (>= 1.3)
- deepTools
- MACS2

### R packages
- dplyr, stringr
- ggplot2, viridis
- GenomicRanges
- DESeq2
- ggpubr, corrplot
- chromVAR
- data.table



## 2. Project Structure and Sample Metadata

Samples follow the naming convention:

`sampleID_antibody_genotype_replicate`

Files Used:
- `80.7_Brg1_KO_rep1.bam`
- `80.10_Brg1_WT_rep1.bam`
- `80.11_Brg1_KO_rep2.bam`
- `80.13_Brg1_WT_rep2.bam`
- `80.1_IgG_KO_rep1.bam`
- `80.18_IgG_WT_rep1.bam`


## 3. Alignment and Quality Control

BAM files were inspected, converted, and aligned.
The following command gives us further information on the bam files:


In [ ]:

samtools view $projPath/alignment/bam/${sampleID}_${antibody}_${genotype}_${replicate}.bam | less -S

Investigation on the previous step showed that 80.11 bam file was truncated, so we converted the corresponding cram file into bam format:

In [ ]:
samtools view -b -T $projPath/alignment/mm10/mm10.fa -o 
$projPath/alignment/bam/${sampleID}_${antibody}_${genotype}_${replicate}.bam  
$projPath/alignment/cram/${sampleID}_${antibody}_${genotype}_${replicate}.cram

Convert bam into fastq file format:

In [ ]:
bedtools bamtofastq -i $projPath/alignment/bam/${sampleID}_${antibody}_${genotype}_${replicate}.bam \
                      -fq $projPath/data/fastq_files/${sampleID}_${antibody}_${genotype}_${replicate}_R1.fq \
                      -fq2 $projPath/data/fastq_files/${sampleID}_${antibody}_${genotype}_${replicate}_R2.fq

Run bowtie2 to perform the alignment to the mus musculus genome mm10 and get the summary .txt file:

In [ ]:
cores=8
ref="/path/to/mm10/index/"
bowtie2 --end-to-end --very-sensitive --no-mixed --no-discordant --phred33 -I 10 -X 700 -p ${cores} -x ${ref}/mm10 -1 ${projPath}/data/fastq_files/${sampleID}_${antibody}_${genotype}_${replicate}_R1.fq -2 ${projPath}/data/fastq_files/${sampleID}_${antibody}_${genotype}_${replicate}_R2.fq -S ${projPath}/alignment/sam/${sampleID}_${antibody}_${genotype}_${replicate}_bowtie2.sam &> ${projPath}/alignment/sam/bowtie2_summary/${sampleID}_${antibody}_${genotype}_${replicate}_bowtie2.txt

Convert bam into bed file format:

In [ ]:
bedtools bamtobed -bedpe -i 
$projPath/alignment/bam/${sampleID}_${antibody}_${genotype}_${replicate}.bam >$projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}.bed

Delete "chr. -1 -1" from 80.11:

In [ ]:
grep -v "\." $projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}.bed > $projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}.bed

Keep the read pairs that are on the same chromosome and fragment length less than 1000bp:

In [ ]:
awk '$1==$4 && $6-$2 < 1000 {print $0}' $projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}.bed >$projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}.clean.bed

Only extract the fragment related columns:

In [ ]:
cut -f 1,2,6 $projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}.clean.bed | sort -k1,1 -k2,2n -k3,3n  >$projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}.fragments.bed

Visualize the sequencing depth and alignment results:

In [ ]:
alignResult = c()
for(sample in sampleList2){
  alignRes = read.table(paste0(projPath, "/alignment/sam/bowtie2_summary/", sample, "_bowtie2.txt"), header = FALSE, fill = TRUE)
  alignRate = substr(alignRes$V1[6], 1, nchar(as.character(alignRes$V1[6]))-1)
  sampleInfo = strsplit(sample, "_")[[1]]
  alignResult = data.frame(Sample = paste0(sampleInfo[1], "_", sampleInfo[2], "_", sampleInfo[3], "_", sampleInfo[4]), 
                           Replicate = sampleInfo[4],
                           Genotype = sampleInfo[3],
                           SequencingDepth = alignRes$V1[1] %>% as.character %>% as.numeric, 
                           MappedFragNum_mm10 = alignRes$V1[4] %>% as.character %>% as.numeric + alignRes$V1[5] %>% as.character %>% as.numeric, 
                           AlignmentRate_mm10 = alignRate %>% as.numeric)  %>% rbind(alignResult, .)
}
alignResult %>% mutate(AlignmentRate_mm10 = paste0(AlignmentRate_mm10, "%"))

![](images/1.png)

It seems that 80.11 has a higher sequencing depth and a lower alignment rate indicating that sample quality was compromised, probably due to technical issues, during sample preparation.


## 4. Sequencing Depth and Alignment Metrics

Bowtie2 alignment summaries were parsed to extract:
- Sequencing depth
- Number of mapped fragments
- Alignment rate

ggplot package was used to generate graphs to visualize the sequencing depth and alignment results:


In [ ]:
figA = alignResult %>% ggplot(aes(x = Sample, y = SequencingDepth/1000000, fill = Sample)) +
  geom_boxplot() +
  geom_jitter(aes(color = Genotype), position = position_jitter(0.15)) +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma", alpha = 0.8) +
  scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9) +
  theme_bw(base_size = 13) +
  ylab("Sequencing Depth per Million") +
  xlab("") + 
  ggtitle("A. Sequencing Depth")

figB = alignResult %>% ggplot(aes(x = Sample, y = MappedFragNum_mm10/1000000, fill = Sample)) +
  geom_boxplot() +
  geom_jitter(aes(color = Genotype), position = position_jitter(0.15)) +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma", alpha = 0.8) +
  scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9) +
  theme_bw(base_size = 13) +
  ylab("Mapped Fragments per Million") +
  xlab("") +
  ggtitle("B. Alignable Fragment (mm10)")

figC = alignResult %>% ggplot(aes(x = Sample, y = AlignmentRate_mm10, fill = Sample)) +
  geom_boxplot() +
  geom_jitter(aes(color = Genotype), position = position_jitter(0.15)) +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, end = 0.9, option = "magma", alpha = 0.8) +
  scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9) +
  theme_bw(base_size = 13) +
  ylab("% of Mapped Fragments") +
  xlab("") +
  ggtitle("C. Alignment Rate (mm10)")

ggarrange(figA, figB, figC, ncol = 2, nrow=2, common.legend = TRUE, legend="bottom")

![Quality analysis of alignment on mm10. Plots for A. Sequencing Depth, B. Alignable Rate (mm10) and C. Alignment Rate (mm10)](images/2.png)

Sequencing depth of sample 80.11 is high but the percentage of DNA corresponding to the mm10 genome is significantly lower than the other samples.
This leads us to the conclusion that probably this sample contains a smaller amount of DNA which was read more times. 

This is probably leading to many copies of the adapter dimers which were amplified multiple times but cannot be sequenced in the genome. This is a typical case of a low complexity library.



## 5. Replicate Reproducibility

Genome-wide reproducibility was assessed by:
- Binning the genome into 500 bp windows
- Counting fragments per bin
- Calculating Pearson correlations between samples

It seems there is not very good reproducibility between the replicates for each condition that could be due to technical reasons or biological heterogeneity between the primary cultures of RGCs used.


## 6. Peak Calling

In order to identify the regions of the genome that are significantly overrepresented in our mapped files and constitute the peaks two peak callers were tested:
- *SEACR*
- *MACS2*

First, add "chr" prefix on the first column in bed files:

In [ ]:
awk '{print "chr" $0}' OFS=\t  $projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}.fragments.bed >$projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}_prefix.fragments.bed

Create bedgraph files:

In [ ]:
chromSize="/path/to/mm10.chrom.size"
bedtools genomecov -bg -i $projPath/alignment/bed/${sampleID}_${antibody}_${genotype}_${replicate}_prefix.fragments.bed -g $chromSize > $projPath/alignment/bedgraph/${sampleID}_${antibody}_${genotype}_${replicate}.fragments.bedgraph

Perform peak calling with SEACR:

In [ ]:
seacr=' path/to/SEACR_1.3.sh'
mkdir -p $projPath/peakCalling/SEACR

bash $seacr $projPath/alignment/bedgraph/${sampleID}_${antibody}_${genotype}_${replicate}.fragments.bedgraph \
     $projPath/alignment/bedgraph/<INSERT CONTROL HERE>.fragments.bedgraph \
     non stringent $projPath/peakCalling/SEACR/${sampleID}_${antibody}_${genotype}_${replicate}_seacr_control.peaks

bash $seacr $projPath/alignment/bedgraph/${sampleID}_${antibody}_${genotype}_${replicate}.fragments.bedgraph 0.01 non stringent $projPath/peakCalling/SEACR/${sampleID}_${antibody}_${genotype}_${replicate}_seacr_top0.01.peaks

Calculate number of top 0.01 peaks called:

In [ ]:
peakN = c()
peakWidth = c()
for(sample in sampleList2){
  sampleInfo = strsplit(sample, "_")[[1]]
      peakInfo = read.table(paste0(projPath, "/peakCalling/SEACR/", sample, "_seacr_top0.01.peaks.stringent.bed"), header = FALSE, fill = TRUE)  %>% mutate(width = abs(V3-V2))
      peakN = data.frame(peakN = nrow(peakInfo), Sample = paste0(sampleInfo[1], "_", sampleInfo[2], "_", sampleInfo[3], "_", sampleInfo[4]), Genotype = sampleInfo[3], Replicate = sampleInfo[4]) %>% rbind(peakN, .)
      peakWidth = data.frame(width = peakInfo$width, Sample = paste0(sampleInfo[1], "_", sampleInfo[2], "_", sampleInfo[3], "_", sampleInfo[4]), Genotype = sampleInfo[3], Replicate = sampleInfo[4])  %>% rbind(peakWidth, .)
}
peakN %>% select(Sample, Genotype, Replicate, peakN)

![](images/3.png)

Remove "chr" prefix on the first column in bed files:

In [ ]:
awk '{sub(/^chr/, "", $1); print}' $projPath/peakCalling/SEACR/new_notation/WT_rep2_seacr_top0.01.peaks.stringent.bed > $projPath/peakCalling/SEACR/new_notation/no_prefix/WT_rep2_seacr_top0.01.peaks.stringent_no_prefix.bed

Calculate FRagment proportion in Peaks regions (FRiPs):

In [ ]:
inPeakData = c()
  for(rep in repL){
    for(type in genotypes){
      peakRes = read.table(paste0(projPath, "/peakCalling/SEACR/new_notation/no_prefix/", type, "_", rep, "_seacr_top0.01.peaks.stringent_no_prefix.bed"), header = FALSE, fill = TRUE)
peak.gr = GRanges(seqnames = peakRes$V1, IRanges(start = peakRes$V2, end = peakRes$V3), strand ="*")
      bamFile = paste0(bamDir2, type, "_", rep, ".bam")
      fragment_counts <- getCounts(bamFile, peak.gr, paired = TRUE, by_rg = FALSE, format = "bam")
      inPeakN = counts(fragment_counts)[,1] %>% sum
      inPeakData = rbind(inPeakData, data.frame(inPeakN = inPeakN, Replicate = rep, Genotype = type))
  }
}
frip = left_join(inPeakData, alignResult, by = c("Genotype", "Replicate")) %>% mutate(frip = inPeakN/MappedFragNum_mm10 * 100)
frip %>% select(Genotype, Replicate, SequencingDepth, MappedFragNum_mm10, AlignmentRate_mm10, FragInPeakNum = inPeakN, FRiPs = frip)

![](images/4.png)

Visualization of peaks quality analysis results:

In [ ]:
fig2A = peakN %>% ggplot(aes(x = Sample, y = peakN, fill = Sample)) +
  geom_boxplot() +
  geom_jitter(aes(color = Genotype), position = position_jitter(0.15)) +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, end = 0.55, option = "magma", alpha = 0.8) +
  scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9) +
  theme_bw(base_size = 13) +
  ylab("Number of Peaks") +
  xlab("")

fig2B = peakWidth %>% ggplot(aes(x = "", y = width, fill = Genotype)) +
  geom_violin() +
  facet_grid(Genotype~Sample) +
  facet_grid(~factor(Sample, levels=c("80.10_Brg1_WT_rep1", "80.13_Brg1_WT_rep2", "80.7_Brg1_KO_rep1", "80.11_Brg1_KO_rep2")))+
  scale_fill_viridis(discrete = TRUE, begin = 0.1, end = 0.55, option = "magma", alpha = 0.8) +
  scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9) +
  scale_y_continuous(trans = "log", breaks = c(400, 3000, 22000)) +
  theme_bw(base_size = 13) +
  ylab("Width of Peaks") +
  xlab("")

fig2C = frip %>% ggplot(aes(x = Sample, y = frip, fill = Sample, label = round(frip, 2))) +
  geom_boxplot() +
  geom_jitter(aes(color = Genotype), position = position_jitter(0.15)) +
  scale_fill_viridis(discrete = TRUE, begin = 0.1, end = 0.55, option = "magma", alpha = 0.8) +
  scale_color_viridis(discrete = TRUE, begin = 0.1, end = 0.9) +
  theme_bw(base_size = 13) +
  ylab("% of Fragments in Peaks") +
  xlab("")
ggarrange(fig2A, fig2B, fig2C, ncol = 2, nrow=2, common.legend = TRUE, legend="bottom")

![](images/5.png)

Peak width is similar between the samples and sample 80.11 has a significantly smaller number of peaks.
If we look at the percentage of DNA assigned to peaks, we can see that the normal samples have a similar percentage while 80.11 has a significantly higher percentage. So, while the 80.11 sample had a lower percentage of DNA mapped to the genome, this DNA contained many peaks.
Overall, we conclude that probably this sample had a lower yield due to technical problems during the isolation and processing of DNA from this sample.



## 7. Visualization

Create bigwig files:


In [ ]:
bamCoverage -b $projPath/alignment/bam/${sampleID}_${antibody}_${genotype}_${replicate}.bam -o $projPath/alignment/bigwig/${sampleID}_${antibody}_${genotype}_${replicate}_raw.bw

Visual inspection of SEACR-called peaks in combination with genome coverage bw files reveals that in most cases, SEACR has called multiple false positive peaks. This is probably because the Brg1 genomic binding pattern is a combination of sharp and broad peaks that cannot be fully captured by the SEACR algorithm. For this reason, we also called peaks using the MACS2 package.

To perform peak calling with MACS2 we uploaded the bam files on Galaxy platform which is a scientific workflow, data integration and data and analysis persistence and publishing platform. We used the default options for paired end bam files. The outputs of MACS2 were bed files that contain all peaks called from the corresponding bam files.

Visual inspection of MACS2 called peaks in combination with genome coverage files reveals a tight agreement between the called peaks and the binding of Brg1, further suggesting that MACS2 has greater power in calling peaks from a binding pattern of both sharp and broad peaks, such as the pattern of Brg1 binding.


The representative loci of Brg1 sites lost/reduced in GemC1 KO cells can be seen below:

![](images/6.png)

![](images/7.png)

Next, the representative loci of Brg1 sites gained in GemC1 KO cells:

![](images/8.png)

Peaks ranked by the sum of the signal density in descending order. Heatmap on CUT&RUN peaks ±3 kb before and after the Region Start Length can be seen below:

![](images/9.png)


## 8. Peak-to-Gene Association

As a core component of the SWI/SNF remodeling complex, Brg1 exerts its function by binding to chromatin and regulating accessibility to enhancers and promoters of genes, thus facilitating transcriptional activation. Therefore, to infer Brg1’s function from the genomics sites it binds to, we needed to find the genes nearest to these Brg1-bound sites. 

In order to do so we used the BED files obtained by peak calling to get the gene/region associations and the GREAT algorithm with the following settings:

- Assembly: mm10 (GRCm38)
- Basal plus extension model
- Distal up to 100 kb

WT and KO-specific Brg1-associated genes were identified for downstream pathway analysis.



## 9. WT vs KO Comparison

The genes common among WT and KO samples were obtained by entering  the genes we got from GREAT on Venny, a webtool that creates Venn diagrams from lists provided by the user. The overlap analysis showed that the WT samples have 3,796 common peaks (35.8 % overlap). Sample 80.10 WT had 6709 peaks not overlapping with 80.13wheressample 80.13 contained 98 peaks not overlapping with 80.10. The KO samples had 4,722 overlapping peaks (45.5%) with sample 80.7 KO containing 5,171 unique/non-overlapping sites and sample 80.11 KO containing 491 exclusive sites. Τhe later finding strengthens our assumption that despite the lower percentage of DNA mapped to the genome, its DNA contained many peaks. Due to this fact, we didn’t discard this sample from our analysis. The results are also in agreement with the Pearson correlation graph which shows that KO samples are more closely correlated compared to the WT samples. Overall, the overlap between WT replicates and KO replicates is satisfactory for further proceeding with differential enrichment and pathway analysis.

After getting the 3796 common genes from our WT samples (80.10 and 80.13) and 4722 KO samples (80.7 and 80.11) we created the final Venn diagram that contains the WT exclusive genes, KO exclusive genes and the common genes between these two:


![](images/10.png)

## 10. Integration with publicly available data

Genes obtained by GREAT in Enrichr in order to perform enrichment analysis.

Genomic regions losing Brg1 occupancy were strongly associated with transcriptional
networks governing cell cycle progression and multiciliogenesis, including targets
of E2F family transcription factors and the p53/p73 pathway. These findings are
consistent with previous evidence that GemC1 cooperates with E2F proteins and p73
to drive the multiciliated cell transcriptional program.

Regions gaining Brg1 binding were enriched near promoters of genes
involved in metabolic regulation, stress-response pathways, and DNA damage
checkpoints. The presence of both gained and lost Brg1 binding across shared
regulatory pathways (e.g., E2F, MYC, CREM) suggests widespread transcriptional
deregulation rather than a simple loss of chromatin engagement.

Collectively, these results indicate that GemC1 is required for proper targeting
of the SWI/SNF complex to lineage-specific regulatory regions, and that its loss
leads to chromatin rewiring incompatible with multiciliated cell differentiation.

## 11. Integration with RNA-seq data

A gmt file was created which contained two rows with the exclusive genes on WT and KO samples respectively and loaded it on GseaPreranked option from GSEA. We also used the rnk file that contains the genes extracted from RNA Seq experiment by their log2FoldChange in ascending order.

We chose “No_Collapse” on Collapse/Remap to gene symbols field and 10000 on Max size: exclude larger sets field. All other options are set to default.
The GSEAPreranked output can be seen below.

![](images/11.png)

We can see that the p-value from both gene sets is high so there is no statistical enrichment.

# 12. Analysis of existing CUT&Tag data

For this analysis, we utilized H3K27me3 CUT&Tag data previously generated in the Taraviras lab. CUT&Tag for H3K27me3 was performed in GemC1 WT and KO RGCs, differentiating under the same conditions as the cells from which the CUT&RUN data were derived. 

Peaks were already called for H3K27me3 data and differential binding was performed using DEseq2. We provided the genes from CUT&Tag experiment in Enrichr using all log2FoldChange from DESeq2 data, as this option gave us the most significant results.

Regions losing H3K27me3 were enriched for canonical Polycomb targets and metabolic genes, suggesting disruption of PRC2-dependent repression. In contrast, regions gaining H3K27me3 were associated with genes involved in neuronal differentiation and Hedgehog signaling — pathways critical for early neural specification.



For the integration with RNA-seq data again, a gmt file was created which contained 8 rows with 8 different options that we got after filtering the p-value and log2FoldChange columns of the DESeq2 data from our CUT&Tag experiment and loaded it on the GseaPreranked option from GSEA. Again, we used the rnk file that contained the genes extracted from the RNA Seq experiment by their log2FoldChange in ascending order.

We chose “No_Collapse” on Collapse/Remap to gene symbols field and 10000 on Max size: exclude larger sets field. All other options are set to default.
The gene set that contained all the upregulated genes, without p-value conditions, was the most significant with p-value = 0.005602241:

The GSEAPreranked output can be seen below:




![](images/12.png)

The Enrichment plot of the gene set that contains all the K27me3 sites gained in GemC1 KO cells can be seen below. 

We can see the plot of the Running Enrichment Score (green line) for the gene set as the analysis walks down the ranked gene list. The black lines in the Running Enrichment Score show where the set gene members appear in the ranked list of genes, indicating the leading-edge subset. 

The Ranked list metric shows the value of log2 fold change as we move down the list of ranked genes.

![](images/13.png)

We can see a significant association between gained H3K27me3 and transcriptional downregulation, supporting a functional role for Polycomb retargeting in mediating gene silencing after GemC1 loss.

## 13. Conclusions

We showed that GemC1 loss reshapes SWI/SNF chromatin targeting and it also regulates Polycomb-mediated repression.

Together, these findings support a model in which GemC1 coordinates opposing
chromatin regulatory systems:

- Promotes SWI/SNF recruitment to activate ciliogenesis programs  
- Restricts Polycomb deposition at differentiation-associated loci  

Loss of GemC1 disrupts this balance, leading to:

- Mislocalization of chromatin remodelers  
- Increased deposition of repressive histone marks  
- Transcriptional repression of lineage-defining genes  

Ultimately, these chromatin alterations likely underlie the impaired
differentiation of multiciliated ependymal cells.